In [1]:


%%capture
%load_ext autoreload
%autoreload 2


import numpy as np
import pandas as pd
import os
import sys
from dotenv import load_dotenv
from pathlib import Path
import datetime as d 
load_dotenv()

# get the current directory
current_dir = os.getcwd()

# change the current working directory to the parent directory
os.chdir(os.path.dirname(current_dir))

path = os.getcwd()
parent = os.path.abspath(os.path.join(path, os.pardir))

USRPATH = "/Users/ducjeremyvu/dev/"


sys.path.append(Path().absolute().parent)
sys.path.append("/Users/ducjeremyvu/dev/wspace/nb")
sys.path.append(parent)
sys.path.append(...)

os.getcwd()


# from g_api import gservice


In [2]:
YEAR = 2024
MONTH = 7

# Step 1: get bonus data and get contact data and merge and create file 

how to get the bonus data: 

https://docs.google.com/spreadsheets/d/1434XUVuzcmaEu7WtnRImQO9_uy0NQp27PB9geS79uhA/edit?resourcekey#gid=627741291


there you can find all relevant bonus data

Bonus_raw_template


then copy the sheet by creating a new sheet (there is a right click function for that)

then change the locale to US or UK 
(reason for this is to change floats with commas to dot notation, due to **comma** seperated values (csv) download, which conflict with germans numeric representation)

then download csv and add the path to next cell

In [3]:
path_bonus = "/Users/ducjeremyvu/Downloads/ucmdownload/240806_bonus_korrektur - Bonus_raw_template.csv"

data_bonus = pd.read_csv(path_bonus)

# create list of students that get bonus to get the max sequence number 
list_bonus_user_ids = list(zip(data_bonus["user_id"],data_bonus["start_date"]))
#string_list_bonus_user_ids = ",".join(list_bonus_user_ids)  #make a string list with commas for the ids 

data_bonus

KeyError: 'start_date'

In [4]:
list_bonus_user_ids

[(51139, '29.03.2024'),
 (51139, '31.03.2024'),
 (68020, '29.03.2024'),
 (65073, '29.03.2024'),
 (60058, '29.03.2024'),
 (54727, '29.03.2024'),
 (69488, '29.03.2024'),
 (65398, '29.03.2024'),
 (51139, '29.03.2024'),
 (51139, '31.03.2024'),
 (68020, '31.03.2024'),
 (71809, '31.03.2024'),
 (60058, '31.03.2024'),
 (47448, '31.03.2024')]

In [12]:
from ucmdata.database.psql import Postgres 

from sshtunnel import SSHTunnelForwarder

server = SSHTunnelForwarder(
    ("100.70.82.247",22),
    ssh_username="djv",
    ssh_password="ucmsysadmin",
    remote_bind_address=('100.70.82.247', 5432)
)

server.start()

pg = Postgres()
# params = {"user_ids":}

# Dynamically create placeholders based on the number of user_ids
conditions = ["(user_id = '{}' AND TO_CHAR(ujd.checkin_time::date, 'DD.MM.YYYY') = '{}')".format(uid, date) for uid, date in list_bonus_user_ids]


query= f"""
SELECT
    ja.user_id,
    ujd.sequence,
    ujp.billable_hour,
    TO_CHAR(ujd.checkin_time::date, 'DD.MM.YYYY') as start_date
FROM
    user_job_dates AS ujd
    JOIN job_applications AS ja ON ja.id = ujd.job_application_id
    JOIN user_job_payrolls AS ujp ON ujp.user_job_date_id = ujd.id
WHERE
"""

# Join conditions with AND or OR
query += " OR ".join(conditions)


data_sequence = pg.get_data(query = query)

# data_sequence["VERTNR"] = data_sequence["max"].astype(int)

# data_sequence.to_excel("/Users/ducjeremyvu/Downloads/ucmdownload/1.xlsx")

data_sequence = data_sequence

server.stop()


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.9p1)
INFO:paramiko.transport:Authentication (publickey) successful!


In [62]:
from datetime import datetime

def first_of_month(date_str):
    # Parse the input date string into a datetime object
    date = datetime.strptime(date_str, "%d.%m.%Y")
    
    # Modify the day part to '1' to set it to the first day of the month
    first_day_of_month = date.replace(day=1)
    
    # Convert the datetime object back to a string in the same format
    first_day_of_month_str = first_day_of_month.strftime("%Y-%m-%d")
    
    return first_day_of_month_str

In [63]:
data_wd_input = data_sequence.merge(data_bonus, on=["user_id","start_date"], how="outer").dropna().rename(columns={"sequence":"VERTNR"})

def get_la(x):
    if x.rate == 13.5:
        return "31C"
    if x.rate == 6.75:
        return "310"

data_wd_input["LA"] = data_wd_input.apply(get_la, axis=1)
data_wd_input["AK"] = "201"
data_wd_input["VD_EBETRAG"] = round(data_wd_input["rate"]*data_wd_input["billable_hour"],2)
data_wd_input["VERTNR"] = data_wd_input["VERTNR"].astype(int)
data_wd_input["VD_ZDAT"] = data_wd_input["start_date"].apply(first_of_month)

data_wd_input

,user_id,VERTNR,billable_hour,start_date,PNR,rate,LA,AK,VD_EBETRAG,VD_ZDAT
0,68020,4,6.50,29.03.2024,A68020,13.50,31C,201,87.75,2024-03-01
1,68020,4,6.50,31.03.2024,A68020,6.75,310,201,43.88,2024-03-01
2,54727,5,7.75,29.03.2024,A54727,13.50,31C,201,104.62,2024-03-01
3,69488,6,7.75,29.03.2024,A69488,13.50,31C,201,104.62,2024-03-01
4,47448,20,7.75,31.03.2024,A47448,6.75,310,201,52.31,2024-03-01
5,65398,6,7.75,29.03.2024,A65398,13.50,31C,201,104.62,2024-03-01
6,51139,2,7.50,29.03.2024,A51139,13.50,31C,201,101.25,2024-03-01
7,51139,2,7.50,29.03.2024,A51139,13.50,31C,201,101.25,2024-03-01
8,51139,3,7.50,31.03.2024,A51139,6.75,310,201,50.62,2024-03-01
9,51139,3,7.50,31.03.2024,A51139,6.75,310,201,50.62,2024-03-01


In [59]:
wd_column_order = [
    "SATZART",
    "FUNKTION",
    "MAN",
    "AK",
    "PNR",
    "NAME",
    "VORNAME",
    "VERTNR",
    "LA",
    "VD_ETAGE",
    "VD_ESTD",
    "VD_EFAKT",
    "VD_EBETRAG",
    "KST",
    "KOSTART",    
    "VD_KTR",
    "VD_DAT",
    "VD_ZDAT",
    "VD_HER",
    # "VD_HER_DATE",
    # "VD_PROZ",
    # "VD_KSTB",
    # "VD_USERID",
    # "VD_WERT",
    # "KST2_MAN",
    # "KST2_AK",
    # "VD_KALK",
#    "VD_TXT",
]

def fill_var(dataframe, YEAR, MONTH):
    MONTH = str(MONTH) if len(str(MONTH)) == 2 else "0" + str(MONTH)
    YEAR_SHORT = str(YEAR)[2:4]
    dataframe = dataframe.copy()
    dataframe["MAN"] = "10246"
    dataframe["SATZART"] = "[VARTAB]"
    dataframe["FUNKTION"] = "INSERT"
#    dataframe["LA"] = LA
    dataframe["VD_HER"] = f"2Z{YEAR_SHORT}{MONTH}"
    #dataframe["VD_ZDAT"] = f"{YEAR}-{MONTH}-01"
    dataframe["VD_DAT"] = ""
    dataframe["VD_ETAGE"] = ""
    dataframe["VD_EBETRAG"] = dataframe["VD_EBETRAG"].astype(str).str.replace(".",",")
    dataframe["KST"] = ""
    dataframe["KOSTART"] = ""
    dataframe["VD_KTR"] = ""
    dataframe["VD_ESTD"] = ""
    dataframe["VD_EFAKT"] = ""
    dataframe["NAME"] = ""
    dataframe["VORNAME"] = "" 
    dataframe["VD_HER_DATE"] = ""
    dataframe["VD_PROZ"] = ""
    dataframe["VD_KSTB"] = ""
    dataframe["VD_USERID"] = ""
    dataframe["VD_WERT"] = ""
    dataframe["KST2_MAN"] = ""
    dataframe["KST2_AK"] = ""
    dataframe["VD_KALK"] = ""
#    dataframe["VD_TXT"] = dataframe["text"]



    return dataframe[wd_column_order]

In [60]:
pd.options.display.max_columns = 100



output = fill_var(data_wd_input, YEAR, MONTH)

output

,SATZART,FUNKTION,MAN,AK,PNR,NAME,VORNAME,VERTNR,LA,VD_ETAGE,VD_ESTD,VD_EFAKT,VD_EBETRAG,KST,KOSTART,VD_KTR,VD_DAT,VD_ZDAT,VD_HER
0,[VARTAB],INSERT,10246,201,A68020,,,4,31C,,,,"87,75",,,,,01.03.2024,2Z2404
1,[VARTAB],INSERT,10246,201,A68020,,,4,310,,,,"43,88",,,,,01.03.2024,2Z2404
2,[VARTAB],INSERT,10246,201,A54727,,,5,31C,,,,"104,62",,,,,01.03.2024,2Z2404
3,[VARTAB],INSERT,10246,201,A69488,,,6,31C,,,,"104,62",,,,,01.03.2024,2Z2404
4,[VARTAB],INSERT,10246,201,A47448,,,20,310,,,,"52,31",,,,,01.03.2024,2Z2404
5,[VARTAB],INSERT,10246,201,A65398,,,6,31C,,,,"104,62",,,,,01.03.2024,2Z2404
6,[VARTAB],INSERT,10246,201,A51139,,,2,31C,,,,"101,25",,,,,01.03.2024,2Z2404
7,[VARTAB],INSERT,10246,201,A51139,,,2,31C,,,,"101,25",,,,,01.03.2024,2Z2404
8,[VARTAB],INSERT,10246,201,A51139,,,3,310,,,,"50,62",,,,,01.03.2024,2Z2404
9,[VARTAB],INSERT,10246,201,A51139,,,3,310,,,,"50,62",,,,,01.03.2024,2Z2404


In [61]:
filename = f"{YEAR}_{MONTH}_zuschlaege_manual_student_service"

output.to_csv("/Users/ducjeremyvu/Downloads/ucmdownload/" + filename + ".csv", index=False, sep=";")